# Example of SSVEP estimators 

In [1]:
from moabb.datasets import SSVEPExo
from moabb.paradigms import SSVEP
import numpy as np
from scipy.stats import norm
from sklearn.pipeline import make_pipeline

from utils import extract_epochs
from estimators import SSVEPCovariance, SSVEPPotato, Potato

paradigm = SSVEP(n_classes=4)
dataset = SSVEPExo()
data = dataset.get_data()

Choosing the first 4 classes from all possible events


In [2]:
# dataset 
subject = 12 
run_train = 'run_0'
run_test = 'run_1'

# epoch onset/offset
tmin = 1
tmax = 7

# for 4 classes 
event_id = {'rest': 1, '13': 2, '17': 3, '21': 4}

Xtrain, ytrain = extract_epochs(data[subject]["session_0"][run_train], event_id=event_id, tmin=tmin, tmax=tmax)
Xtest, ytest = extract_epochs(data[subject]["session_0"][run_test], event_id=event_id, tmin=tmin, tmax=tmax)


In [5]:
fs = 256
# parameters from SSVEPCovariance
flickering_frequencies = (13, 17, 21)
band_width = 4
order = 2
filter_design = 'iir'
padlen = None


ssvep_covariance = SSVEPCovariance(fs, flickering_frequencies,'scm', padlen  = padlen)
ssvep_potato = SSVEPPotato()
potato = Potato()

# Rejection using single potato 
reject_pipeline_1 = make_pipeline(ssvep_covariance, potato)
z_score_1 = reject_pipeline_1.fit(Xtrain).transform(Xtest)
z_to_p = lambda z_score: 1 - norm.cdf(z_score)
p_score_1 = np.apply_along_axis(z_to_p, 0, z_score_1)
print(f'Test SQI using single potato :\n {p_score_1}')

# Rejection using potato class field 
reject_pipeline_2 = make_pipeline(ssvep_covariance, ssvep_potato)
reject_pipeline_2.fit(Xtrain, ytrain)
print(f'Train trial SQI using potatoes class field :\n {reject_pipeline_2.transform(Xtrain)}')
print(f'Test SQI using potatoes class field :\n {reject_pipeline_2.transform(Xtest)}')


Test SQI using single potato :
 [6.72255335e-02 2.12278774e-01 5.06814707e-02 1.03714916e-04
 1.01493410e-02 2.27071697e-03 2.26257946e-02 1.37862027e-02
 7.72364915e-03 6.70508787e-02 3.42590481e-04 3.07465040e-02
 8.16118721e-02 1.06190849e-03 8.31448844e-03 9.43464025e-03
 1.70319881e-03 7.59240234e-02 4.00220811e-03 2.27175135e-03
 4.81873960e-02 3.62458187e-03 1.05403270e-01 1.72401879e-03
 4.44148872e-03 1.91125270e-05 3.84664583e-03 3.06322114e-03
 3.36188307e-04 6.92967426e-07 8.49139628e-04 9.32752124e-04]
Test SQI using potatoes class field :
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
